# Consolidacion de datos de F1

La idea de este archivo es probar el flujo de las funciones creadas. Para traer los datos de la API, preprocesarlos y tener una tabla final consolidada con las distintas fuentes de datos.

In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
# Module imports
from funcs_data_extraction import *
import pandas as pd
import numpy as np

## 1. Extraccion de datos de resultados de Qualifiers y de Carreras

**Definicion de parametros iniciales del modelo**

In [19]:
# Se calculara los resultados con las 10 carreras anteriores.
season_years = [2022, 2023]
round_wdw = 10
prediction_round = 11 # si es inferior a 10, crear flujo para las carreras
# de la ronda anterior

### 1.1 Extraccion y transformacion de datos de Qualifiers
**PENDIENTES**:
- Falta terminar En caso de que se requiera la informacion de temporadas distintas

**Obtener la tabla preprocesada de los tiempos de Qualis**

In [23]:
# En caso de que la ronda que se quiera predecir necesite de la cantidad de datos de la misma temporada
if prediction_round > round_wdw:
    # Obtencion de todos los datos de una temporada
    max_season = max(season_years)
    print('Collecting  data from season {temp}...'.format(temp=max_season))
    season_data = get_season_qualifying_results(max_season)
    print('Data collection from season {temp} successful!'.format(temp=max_season))
    
    print('Concatenating data...')
    # Se buscan los datos de las N rondas anteriores a la ronda que se quiere predecir
    # N es la cantidad de rondas  
    for i in range(prediction_round-round_wdw, prediction_round):
        round_df = get_round_qualifyer_results(season_data, i)
        if i == prediction_round-round_wdw:
            all_qualis_df = round_df
            break # Prueba para los datos de la primera ronda, quitar esta linea para todos los datos anteriores
        else:
            all_qualis_df = pd.concat([all_qualis_df, round_df], axis=0)
    print('Data from all rounds concatenated!')

Data collection from season 2023 successful!
Concatenating data...
Data from all rounds concatenated!


In [25]:
len(season_data)

22

In [7]:
print('Preprocessing all rounds data...')
qualis_df = preprocess_round_data(all_qualis_df)
print('Preprocessing successful')

Preprocessing all rounds data...
Preprocessing successful


In [8]:
# Marcar con uno o 0 si el piloto paso califico de Q1 a Q2 y de Q2 a Q3
#df['new_column'] = np.where(df['best_q2'].isna(), 0, 1)
qualis_df['qualifies_to_Q2'] = np.where(qualis_df['best_time_Q2'].isna(), 0, 1)
qualis_df['qualifies_to_Q3'] = np.where(qualis_df['best_time_Q3'].isna(), 0, 1)

### 1.2 Extraccion y transformacion de datos de Carreras

## Pendientes:
- Falta agregar los datos de la carrera para cada ronda
- Falta agregar datos de la ronda de la cual se predice resultados, incluye los datos de Qualis y datos de la carrera

In [9]:
if prediction_round > round_wdw:
    # Obtencion de todos los datos de carrera de una temporada
    max_season = max(season_years)
    print('Collecting  data from season {temp}...'.format(temp=max_season))
    season_race_data = get_season_race_results(max_season)
    print('Data collection from season {temp} successful!'.format(temp=max_season))
    print('Concatenating data...')

Data collection from season 2023 successful!
Concatenating data...


In [11]:
season_race_data

[]

## 2. Analisis Exploratorio de Datos
**Realizar EDA en otro notebook**

Explorar 2 relaciones especificamente:
- Relacion de los resultados de la quali de la misma ronda con los resultados de la carrera de esa ronda
- Relacion entre los resultados de rondas anteriores con los de la carrera de la ronda a predecir.